# AIDA Freediving Records

The Scope of that project is explore and display all freediving records from one of the world organization of freediving, AIDA in a series of D3.js visualizations. Such plateform could be extended to other datasets that share similar types of information (athlete name, country, record values (time, distance), location of the event, date).

1- Scraping the data from the website

2- Data preparation / cleaning / extension (separate name / country, get GPS locations, get gender...)

3- Early data exploration

4- D3.js data visualizations

In [9]:
from bs4 import BeautifulSoup
from lxml import html
import requests as rq
import pandas as pd
import re
import logging

In [10]:
def get_discipline_value(key):
    '''This function selects one of 6 disciplines (dictionary keys) and allocates its corresponding value (id)
    to discipline_url. The function is called in scraper() function to obtain corresponding html pages'''

    disc = {'STA': 8 ,
        'DYN': 6,
        'DNF': 7,
        'CWT': 3,
        'CNF': 4,
        'FIM': 5
        }
    if key in disc.keys():
        value = disc[key]
        discipline_url = '{}{}'.format('&disciplineId=', value) 
        return discipline_url
    else:
        print 'Check your spelling. ' + key + ' is not a freediving discipline'
        

In [11]:
get_discipline_value('NFT')

Check your spelling. NFT is not a freediving discipline


In [52]:
def scraper(key):
    ''' This function crawls through an entire freediving discipline, regardless how many pages it consists of. '''

    '''Obtain html code for url and Parse the page'''
    base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='
    p = 1
    url = '{}{}{}'.format(base_url, p, get_discipline_value(key))
    page = rq.get(url)
    soup = BeautifulSoup(page.content, "lxml")


    '''Use regex to identify the maximum number of pages for the discipline of interest'''
    page_count = soup.findAll(text=re.compile(r"Page .+ of .+"))
    max_pages = str(page_count).split(' ')[3].split('\\')[0]
    
    data = []
    while p < 3 :
         
        '''For each page, create corresponding url, request the library, obtain html code and parse the page'''
        url = '{}{}{}'.format(base_url, p, get_discipline_value(key))

        '''The break plays the role of safety guard if dictionary key is wrong (not spelled properly or non-existent) then the request
        for library is not executed (and not going through the for loop to generate the data), an empty dataframe is saved'''
        if url == '{}{}None'.format(base_url, p):
            break
        else:
            new_page = rq.get(url)
            new_soup = BeautifulSoup(new_page.content, "lxml")

            '''For each page, each parsed page is saved into the list named "data" '''
            rows = new_soup.table.tbody.findAll('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols if ele]) 
            
            p += 1
    
    '''Results from list "data" are saved in a dataframe df '''
    df=pd.DataFrame(data)
    logging.warning('Finished!')

    '''Dataframe df is saved in file results.txt to access results offline'''
    filename = '/Users/fabienplisson/Desktop/DeepPlay/deepplay/notebooks/results_{}.csv'.format(key)
    with open(filename, 'a') as f:
        f.write(str(df))
    f.closed


In [46]:
scraper('FIM')

In [57]:
df = pd.read_csv('./results_FIM.csv')
df

CParserError: Error tokenizing data. C error: Expected 1 fields in line 79, saw 4


In [54]:
def cleaner(dataframe):
    dataframe.columns = ["Ranking", "Name", "Results", "Announced", "Points", "Penalties", "Date", "Place"]
    dataframe.set_index('Ranking')
    return dataframe

In [55]:
cleaner(df)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 8 elements